# Triplet Loss with Faces -- Various tests

## Initialization

In [ ]:
!pip install torch

In [ ]:
!pip install mat73

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
!git clone https://github.com/mgornet/CNPEN

### Check device

In [ ]:
!nvidia-smi

### Import Librairies

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from time import perf_counter
from typing import Callable
import itertools
import mat73
import pandas as pd
import re

import sys
import os
import tarfile

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

import os.path as op
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve
    
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, auc, roc_curve, \
precision_recall_curve, accuracy_score, f1_score
from scipy import optimize

In [ ]:
%cd ./CNPEN/files/

from triplet import TripletGenerator, TripletLearner, TripletLoss, TripletLossRaw, distance
from builder import create_dataframe, from_tensor_to_numpy, from_numpy_to_tensor, extend_dataframe
from prints import print_img, print_img_from_path, print_img_from_id, \
print_img_from_classid, print_from_gen, print_from_gen2, print_pair, print_hist_loss, \
print_hist_dist, print_hist_dist_zoom, print_img_category, \
print_roc, print_logistic_regression, print_prec_recall
from test_train_loops import training, testing, adaptative_train, compute_distances
from classification import authentification_img, predict, build_df_fairness, \
triplet_acc

In [ ]:
!pwd

## Generate Data

### Create dataframe

In [ ]:
URL = "http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz"
FILENAME = "lfw-deepfunneled.tgz"

if not op.exists(FILENAME):
    print('Downloading %s to %s...' % (URL, FILENAME))
    urlretrieve(URL, FILENAME)

if not op.exists("lfw"):
    print('Extracting image files...')
    tar = tarfile.open("lfw-deepfunneled.tgz")
    tar.extractall("lfw")
    tar.close()

In [ ]:
PATH = "lfw/lfw-deepfunneled/"

In [ ]:
tic = perf_counter()
df_init, all_imgs = create_dataframe()
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

In [ ]:
tic = perf_counter()
df = extend_dataframe(df_init)
toc = perf_counter()
print(f"DataFrame extention: {((toc - tic)/60):.1f} min")

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.columns

### Build sets, generators and network

In [ ]:
num_classes = len(df.Classid.unique())
print("Number of individuals: ", num_classes)

In [ ]:
indiv_min = df.Classid.min()
split_train_valid = int(num_classes * 0.75)
split_train_test = int(num_classes * 0.8)
indiv_max = df.Classid.max()

In [ ]:
print(f"Train set from indiv {indiv_min} to {split_train_valid-1}")
print(f"Valid set from indiv {split_train_valid} to {split_train_test-1}")
print(f"Test set from indiv {split_train_test} to {indiv_max}")

In [ ]:
df_train = df[df.Classid<split_train_valid]
df_valid = df[(df.Classid>=split_train_valid)&(df.Classid<split_train_test)]
df_test = df[df.Classid>=split_train_test]

In [ ]:
print("Number of training images: ", len(df_train))
print("Number of validation images: ", len(df_valid))
print("Number of testing images: ", len(df_test))
print("Number of total images: ", len(df_train)+len(df_valid)+len(df_test))
print("len original: ", len(df))

In [ ]:
print("Number of individuals in the training set: ", len(df_train.Classid.unique()))
print("Number of individuals in the validation set: ", len(df_valid.Classid.unique()))
print("Number of individuals in the testing set: ", len(df_test.Classid.unique()))

In [ ]:
value_count = df_train.Classid.value_counts()
print("Number of individuals with more than one image in the training set: ", len(value_count[value_count.values>1]))
value_count = df_valid.Classid.value_counts()
print("Number of individuals with more than one image in the validation set: ", len(value_count[value_count.values>1]))
value_count = df_test.Classid.value_counts()
print("Number of individuals with more than one image in the testing set: ", len(value_count[value_count.values>1]))

In [ ]:
BATCH_SIZE = 128 # 128
BATCH_VALID_SIZE = 128 #128 #8
BATCH_TEST_SIZE = 128 #128 #32

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TripletLearner(base_channels=32, dropout=0)
model.to(device)

lr = 1e-3/2 # 1e-3/2
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[300,400,500,600,700,800,900], gamma=0.5)

margin = 0.2
criterion = TripletLoss(margin)
criterion_test = TripletLossRaw(margin)

epochs = 1000

In [ ]:
gen_train = TripletGenerator(df_train, all_imgs, BATCH_SIZE, device, model, margin, transform = True)
train_loader = DataLoader(gen_train, batch_size=None, shuffle=True)

gen_valid = TripletGenerator(df_valid, all_imgs, BATCH_VALID_SIZE, device, model, margin)
valid_loader = DataLoader(gen_valid, batch_size=None, shuffle=True)

gen_test = TripletGenerator(df_test, all_imgs, BATCH_TEST_SIZE, device, model, margin)
test_loader = DataLoader(gen_test, batch_size=None, shuffle=True)

### Verify shapes

In [ ]:
# Shape of the generator

print(len(gen_train))
print(len(gen_train[0]))
print(gen_train[0][0].shape)

In [ ]:
# Shape of the loader

print(len(train_loader))
print(train_loader)

## Test printing functions

### Basic test prints

In [ ]:
# Print an image from its path

print_img_from_path(df.iloc[0].Path)

In [ ]:
# Print an image from a certain identity

print_img_from_classid(df,7)

In [ ]:
# Print a few images from the train generator

print_from_gen(gen_train,0)

In [ ]:
# Print a few images from the test generator

print_from_gen(gen_test,0)

In [ ]:
# Print all the images of a batch from the train generator (warning: will print a lot of images)

# print_from_gen2(gen_valid)

### Test Augmentation

In [ ]:
xa,xp,xn = gen_test[0]

In [ ]:
x = gen_train.apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([
            transforms.RandomRotation((-5,5))],
        p=0.95),
        transforms.RandomApply([
            transforms.RandomResizedCrop(size=60, scale=(0.7,0.7), ratio=(1.,1.))],
        p=0.95),
        # transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
    ]
)

In [ ]:
x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([transforms.RandomRotation((-5,-5))])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([transforms.RandomHorizontalFlip(p=1)])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([transforms.RandomResizedCrop(size=60, scale=(0.8,1.1), ratio=(1.,1.))])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([transforms.RandomResizedCrop(size=60, scale=(0.8,0.8))])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25)])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([transforms.Normalize(mean=[0,0,0],std=[255,255,255])])

x = apply_augmentation(xa[0])
print(f'Original image: between {xa[0].min()} and {xa[0].max()}')
print_img(from_tensor_to_numpy(xa[0])/255)
print(f'Normalized image: between {x.min()} and {x.max()}')
print_img(from_tensor_to_numpy(x))

apply_augmentation_inverse = transforms.Compose([transforms.Normalize(mean=[0,0,0],std=[1/255,1/255,1/255])])

y = apply_augmentation_inverse(x)
print(f'Back to original image: between {y.min()} and {y.max()}')
print_img(from_tensor_to_numpy(y)/255)

In [ ]:
apply_augmentation = transforms.Compose([
    transforms.Normalize(mean=[0,0,0],std=[255,255,255]),
    transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25),
    transforms.Normalize(mean=[0,0,0],std=[1/255,1/255,1/255])
])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

In [ ]:
apply_augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([
        transforms.RandomResizedCrop(size=60, scale=(0.8,1.), ratio=(0.95,1.05))],
    p=0.9),
    transforms.RandomApply([
        transforms.Compose([
            transforms.Normalize(mean=[0,0,0],std=[255,255,255]),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            transforms.Normalize(mean=[0,0,0],std=[1/255,1/255,1/255])
        ]),],
    p=0.1),
    transforms.RandomApply([
        transforms.RandomRotation((-5,5))],
    p=0.7),
])

x = apply_augmentation(xa[0])
print_img(from_tensor_to_numpy(xa[0])/255)
print_img(from_tensor_to_numpy(x)/255)

### Test Categories

#### Supposed race

In [ ]:
print_img_category(all_imgs, df, 'White')

In [ ]:
print_img_category(all_imgs, df, 'White', inverse=True)

In [ ]:
print_img_category(all_imgs, df, 'Black')

In [ ]:
print_img_category(all_imgs, df, 'Asian')

In [ ]:
print_img_category(all_imgs, df, 'Indian')

#### Supposed gender

In [ ]:
print_img_category(all_imgs, df, 'Male')

In [ ]:
print_img_category(all_imgs, df, 'Male', inverse=True)

#### Supposed age category

In [ ]:
print_img_category(all_imgs, df, 'Baby')

In [ ]:
print_img_category(all_imgs, df, 'Baby', inverse=True)

In [ ]:
print_img_category(all_imgs, df, 'Child')

In [ ]:
print_img_category(all_imgs, df, 'Youth')

In [ ]:
print_img_category(all_imgs, df, 'Middle Aged')

In [ ]:
print_img_category(all_imgs, df, 'Senior')

#### Accessories

In [ ]:
print_img_category(all_imgs, df, 'Wearing Hat')

In [ ]:
print_img_category(all_imgs, df, 'Eyeglasses')

In [ ]:
print_img_category(all_imgs, df, 'Sunglasses')

In [ ]:
print_img_category(all_imgs, df, 'Sunglasses', inverse=True)

#### Hair(s)

In [ ]:
print_img_category(all_imgs, df, 'No Beard')

In [ ]:
print_img_category(all_imgs, df, 'No Beard', inverse=True)

In [ ]:
print_img_category(all_imgs, df, 'Mustache')

## Other tests

### Explore df

In [ ]:
df.describe()

In [ ]:
value_count = df.Classid.value_counts()
value_count

In [ ]:
df.loc[1871]

In [ ]:
print_img(from_tensor_to_numpy(all_imgs[1871])/255)

### Test the valid loader

In [ ]:
valid_list = list(valid_loader)
for _ in range(2):
    new_valid = list(valid_loader)
    valid_list[0][0] = torch.cat((valid_list[0][0],new_valid[0][0]), axis=0)
    valid_list[0][1] = torch.cat((valid_list[0][1],new_valid[0][1]), axis=0)
    valid_list[0][2] = torch.cat((valid_list[0][2],new_valid[0][2]), axis=0)

In [ ]:
print_img(from_tensor_to_numpy(valid_list[0][0][78])/255)
print_img(from_tensor_to_numpy(valid_list[0][1][78])/255)
print_img(from_tensor_to_numpy(valid_list[0][2][78])/255)

In [ ]:
print_img(from_tensor_to_numpy(valid_list[0][0][79])/255)
print_img(from_tensor_to_numpy(valid_list[0][1][79])/255)
print_img(from_tensor_to_numpy(valid_list[0][2][79])/255)

### Print the histogram inductive bias

In [ ]:
model = TripletLearner(base_channels=32)
model.eval()

In [ ]:
device = torch.device('cpu')

gen = TripletGenerator(df_train, all_imgs, BATCH_SIZE, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

pos_dist, neg_dist, _ = compute_distances(loader, device, model)

In [ ]:
print_hist_dist_zoom(pos_dist,neg_dist,zoom=3e-3)

### Print the histogram of initial loss

In [ ]:
model = TripletLearner(base_channels=32)
model.eval()

In [ ]:
loss = testing(train_loader, device, model, criterion_test)

In [ ]:
print_hist_loss(loss)

### Print the learning rate schedule

In [ ]:
lrs = []

for epoch in range(1000):
    optimizer.step()
    lrs.append(optimizer.param_groups[0]["lr"])
    scheduler.step()

In [ ]:
plt.figure()
plt.plot(range(1000),lrs, color='red')
plt.xlabel('Number of epochs')
plt.ylabel('Learning rate value')
plt.show()

## Test Classification

In [ ]:
model = TripletLearner(base_channels=32)
model.load_state_dict(torch.load("../models/without_jitter_and_p.pth",map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

In [ ]:
# Build THRESHOLD (for details, see the notebook "determine_threshold")

gen = TripletGenerator(df_valid, all_imgs, BATCH_VALID_SIZE, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

list_loader = []
for _ in range(10):
    list_loader.extend(list(loader))

pos_dist, neg_dist, _ = compute_distances(list_loader, device, model) #loader

y_pos = [1 for _ in range(len(pos_dist))]
y_neg = [0 for _ in range(len(neg_dist))]

y = y_pos + y_neg
X = pos_dist + neg_dist
Xmoins = np.array(X)*(-1)
Xlogistic = np.array(Xmoins).reshape(-1,1)

clf = LogisticRegression(random_state=0).fit(Xlogistic, y)

THRESHOLD = (clf.intercept_/clf.coef_)[0,0]
print("THRESHOLD with logistic regression:", THRESHOLD)

### Test random pairs

In [ ]:
# Print a random pair classified as "different"

print("ARE SUPPOSED TO BE ALWAYS DIFFERENT")

Xa, Xp, Xn = next(iter(loader))

rdm = random.randint(0,len(Xa)-1)

bin = authentification_img(Xa[rdm], Xn[rdm], device, model, THRESHOLD, verbose = True)

print_pair(Xa[rdm], Xn[rdm])

if bin==1 :
    print("Same person")
else :
    print("Different people")

In [ ]:
# Print a random pair classified as "same"

print("ARE SUPPOSED TO BE ALWAYS THE SAME")

Xa, Xp, Xn = next(iter(loader))

rdm = random.randint(0,len(Xa)-1)

bin = authentification_img(Xa[rdm], Xp[rdm], device, model, THRESHOLD, verbose = True)

print_pair(Xa[rdm], Xp[rdm])

if bin==1 :
    print("Prediction: Same person")
else :
    print("Prediction: Different people")

### Print extremes errors

In [ ]:
len(list_loader), len(list_loader[0]), list_loader[0][0].shape

In [ ]:
# Reorganize images to fetch them more easily

img_classif = []
for i in range(3):
    img_classif.append(torch.cat([list_loader[j][i] for j in range(len(list_loader))], 0))

In [ ]:
len(img_classif), img_classif[0].shape

In [ ]:
print(len(img_classif))
print(img_classif[0].shape)

In [ ]:
# Positive pair with hight and low distance

pos_dist2 = pos_dist.copy()
pos_dist3 = pos_dist.copy()

dist_max_list, id_max_list = [], []
dist_min_list, id_min_list = [], []

for _ in range(5) :
    
    dist_max, id_max = max(pos_dist2), np.argmax(np.array(pos_dist2))
    dist_max_list.append(dist_max)
    id_max_list.append(id_max)
    pos_dist2.pop(id_max)
    
    dist_min, id_min = min(pos_dist3), np.argmin(np.array(pos_dist3))
    dist_min_list.append(dist_min)
    id_min_list.append(id_min)
    pos_dist3.pop(id_min)

In [ ]:
print("SAME IDENTITY WITH HIGH DISTANCES")
print("MORE LIKELY TO BE FALSE NEGATIVES")
#  TRUE SAME ID WITH HIGH PREDICTED DISTANCES

for dist,id_img in zip(dist_max_list,id_max_list):
    print("Distance:", dist)
    anchor = img_classif[0][id_img]
    positive = img_classif[1][id_img]
    print_pair(anchor, positive)

In [ ]:
print("SAME IDENTITY WITH LOW DISTANCES")
print("MORE LIKELY TO BE TRUE POSITIVES")

for dist,id_img in zip(dist_min_list,id_min_list):
    print("Distance:", dist)
    anchor = img_classif[0][id_img]
    positive = img_classif[1][id_img]
    print_pair(anchor, positive)

In [ ]:
# Negative pair with hight and low distance

neg_dist2 = neg_dist.copy()
neg_dist3 = neg_dist.copy()

dist_max_list, id_max_list = [], []
dist_min_list, id_min_list = [], []

for _ in range(5) :
    
    dist_max, id_max = max(neg_dist2), np.argmax(np.array(neg_dist2))
    dist_max_list.append(dist_max)
    id_max_list.append(id_max)
    neg_dist2.pop(id_max)
    
    dist_min, id_min = min(neg_dist3), np.argmin(np.array(neg_dist3))
    dist_min_list.append(dist_min)
    id_min_list.append(id_min)
    neg_dist3.pop(id_min)

In [ ]:
print("DIFFERENT IDENTITIES WITH LOW DISTANCES")
print("MORE LIKELY TO BE FALSE POSITIVES")

for dist,id_img in zip(dist_min_list,id_min_list):
    print("Distance:", dist)
    anchor = img_classif[0][id_img]
    negative = img_classif[2][id_img]
    print_pair(anchor, negative)

In [ ]:
print("DIFFERENT IDENTITIES WITH HIGH DISTANCES")
print("MORE LIKELY TO BE TRUE NEGATIVES")

for dist,id_img in zip(dist_max_list,id_max_list):
    print("Distance:", dist)
    anchor = img_classif[0][id_img]
    negative = img_classif[2][id_img]
    print_pair(anchor, negative)

### Fairness

In [ ]:
# Build THRESHOLD (for details, see the notebook "determine_threshold")

gen = TripletGenerator(df_valid, all_imgs, BATCH_VALID_SIZE, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

list_loader = []
for _ in range(10):
    list_loader.extend(list(loader))

pos_dist, neg_dist, _ = compute_distances(list_loader, device, model) #loader

y_pos = [1 for _ in range(len(pos_dist))]
y_neg = [0 for _ in range(len(neg_dist))]

y = y_pos + y_neg
X = pos_dist + neg_dist
Xmoins = np.array(X)*(-1)
Xlogistic = np.array(Xmoins).reshape(-1,1)

clf = LogisticRegression(random_state=0).fit(Xlogistic, y)

THRESHOLD = (clf.intercept_/clf.coef_)[0,0]
print("THRESHOLD with logistic regression:", THRESHOLD)

In [ ]:
gen = TripletGenerator(df_test, all_imgs, BATCH_TEST_SIZE, device, model, margin, return_id=True)
tic = perf_counter()
df_fairness = build_df_fairness(all_imgs, df_test, gen, 20, device, model, THRESHOLD)
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

In [ ]:
my_img = all_imgs[df_fairness[df_fairness.A_Male==0].id_A.iloc[3]]

In [ ]:
print_img(from_tensor_to_numpy(my_img)/255)

In [ ]:
my_img = all_imgs[list(df_fairness[df_fairness.A_Male==0].id_A)]

In [ ]:
for k in range(10):
    print_img(from_tensor_to_numpy(my_img[k])/255)

## My images

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TripletLearner(base_channels=32, dropout=0)
model.to(device)

lr = 1e-3/2 # 1e-3/2
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[300,400,500,600,700,800,900], gamma=0.5)

margin = 0.2
criterion = TripletLoss(margin)
criterion_test = TripletLossRaw(margin)

epochs = 1000

In [ ]:
model = TripletLearner(base_channels=32)
model.load_state_dict(torch.load("./models/without_jitter_and_p.pth",map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

In [ ]:
MY_IMGS_PATH = '../my_imgs/'

In [ ]:
df_my_imgs, all_my_imgs = create_dataframe(MY_IMGS_PATH)

In [ ]:
print_img(from_tensor_to_numpy(all_my_imgs[0])/255)

In [ ]:
# for i in range(len(all_my_imgs)):
#     print_img(from_tensor_to_numpy(all_my_imgs[i])/255)

In [ ]:
df_my_imgs

In [ ]:
print_img(from_tensor_to_numpy(all_my_imgs[19]/255))

In [ ]:
gen = TripletGenerator(df_my_imgs, all_my_imgs, 128, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

In [ ]:
Xa, Xp, Xn = next(iter(loader))

rdm = random.randint(0,len(Xa)-1)

bin = authentification_img(Xa[rdm], Xp[rdm], device, model, THRESHOLD, verbose = True)

if bin==1 :
    print("Prediction: Same person")
else :
    print("Prediction: Different people")
    
print_pair(Xa[rdm], Xp[rdm])

In [ ]:
Xa, Xp, Xn = next(iter(loader))

rdm = random.randint(0,len(Xa)-1)

bin = authentification_img(Xa[rdm], Xn[rdm], device, model, THRESHOLD, verbose = True)

if bin==1 :
    print("Prediction: Same person")
else :
    print("Prediction: Different people")
    
print_pair(Xa[rdm], Xn[rdm])

In [ ]:
def print_all_pairs_with_distances(df, all_imgs):
    list_paths = list(df.index)
    pos_pairs = list(itertools.combinations(list_paths,2))
    random.shuffle(pos_pairs)
    
    for pair in range(len(list_paths)):
        i,j = pos_pairs[pair]
        img1 = all_imgs[i]
        img2 = all_imgs[j]
        bin = authentification_img(img1, img2, device, model, THRESHOLD, verbose = True)

        if bin==1 :
            print("Prediction: Same person")
        else :
            print("Prediction: Different people")
            
        print_pair(img1, img2)

In [ ]:
print_all_pairs_with_distances(df_my_imgs[df_my_imgs.Classid==0], all_my_imgs)

In [ ]:
print_all_pairs_with_distances(df_my_imgs[df_my_imgs.Classid==1], all_my_imgs)

In [ ]:
print_all_pairs_with_distances(df_my_imgs[df_my_imgs.Classid==2], all_my_imgs)

### Precision with my images

In [ ]:
list_loader = []
for _ in range(50):
    list_loader.extend(list(loader))

In [ ]:
pos_dist, neg_dist, _ = compute_distances(list_loader, device, model) #loader

In [ ]:
print_hist_dist_zoom(pos_dist,neg_dist)

In [ ]:
y_pos = [1 for _ in range(len(pos_dist))]
y_neg = [0 for _ in range(len(neg_dist))]

y = y_pos + y_neg
X = pos_dist + neg_dist

Xmoins = np.array(X)*(-1)

fpr_dist, tpr_dist, thresholds_dist = roc_curve(y, Xmoins)
roc_auc_dist = auc(fpr_dist,tpr_dist)

precision_dist, recall_dist, thresholds_recall_dist = precision_recall_curve(y, Xmoins)
auc_s_dist = auc(recall_dist, precision_dist)

In [ ]:
print_roc(fpr_dist, tpr_dist, roc_auc_dist)
print_prec_recall(precision_dist, recall_dist, auc_s_dist)

In [ ]:
# Predict classification from THRESHOLD

y_pred_pos_bins, y_pred_neg_bins = predict(list_loader, device, model, THRESHOLD) #loader
y_pred_bins = y_pred_pos_bins + y_pred_neg_bins
triplet_accuracy = triplet_acc(list_loader, device, model)

print("Confusion Matrix Total")
print(confusion_matrix(y, y_pred_bins))
print("\n")
print("Accuracy score:",accuracy_score(y, y_pred_bins))
print("Triplets where d(A,P) < d(A,N): ",round(100*triplet_accuracy,1),"%")

### Precision with test images

In [ ]:
gen = TripletGenerator(df_test, all_imgs, BATCH_TEST_SIZE, device, model, margin)
loader = DataLoader(gen, batch_size=None, shuffle=True)

list_loader = []
for _ in range(10):
    list_loader.extend(list(loader))

pos_dist, neg_dist, _ = compute_distances(list_loader, device, model) #loader

y_pos = [1 for _ in range(len(pos_dist))]
y_neg = [0 for _ in range(len(neg_dist))]

y = y_pos + y_neg
X = pos_dist + neg_dist
Xmoins = np.array(X)*(-1)

fpr_dist, tpr_dist, thresholds_dist = roc_curve(y, Xmoins)
roc_auc_dist = auc(fpr_dist,tpr_dist)

precision_dist, recall_dist, thresholds_recall_dist = precision_recall_curve(y, Xmoins)
auc_s_dist = auc(recall_dist, precision_dist)

In [ ]:
print_hist_dist_zoom(pos_dist,neg_dist)

In [ ]:
print_roc(fpr_dist, tpr_dist, roc_auc_dist)
print_prec_recall(precision_dist, recall_dist, auc_s_dist)

In [ ]:
# Predict classification from THRESHOLD

y_pred_pos_bins, y_pred_neg_bins = predict(list_loader, device, model, THRESHOLD) #loader
y_pred_bins = y_pred_pos_bins + y_pred_neg_bins
triplet_accuracy = triplet_acc(list_loader, device, model)

print("Confusion Matrix Total")
print(confusion_matrix(y, y_pred_bins))
print("\n")
print("Accuracy score:",accuracy_score(y, y_pred_bins))
print("Triplets where d(A,P) < d(A,N): ",round(100*triplet_accuracy,1),"%")